In [1]:
# Disable GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
import time

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np
from itertools import combinations
from enum import Enum

In [4]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
from pkg.ros_rviz import *

In [5]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *
from pkg.utils import *
from pkg.ur10 import *
from pkg.geometry import *
from pkg.collision import *
from pkg.distance import *
from pkg.distance_calculator import *
from pkg.binding_calculator import *
from pkg.graph_tf import *

In [6]:
def get_by_name(item_list, name):
    return [item for item in item_list if item.name==name][0]

In [7]:
def get_box_binding(box, btype=BindingType.SUCC, hexahedral=False):
    box_name, dims = box.name, box.dims
    Xhalf, Yhalf, Zhalf = np.array(dims)/2
    bindings = [BindingInfo(
                    name=box_name+'-top', btype=btype, obj_name=box_name, 
                    gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,0), (0,0,Zhalf))), 
                BindingInfo(
                    name=box_name+'-bottom', btype=btype, obj_name=box_name, 
                    gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,np.pi), (0,0,-Zhalf)))]
    if hexahedral:
        bindings += [
            BindingInfo(
                name=box_name+'-right', btype=btype, obj_name=box_name, 
                gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,np.pi/2,0), (Xhalf,0,0))), 
            BindingInfo(
                name=box_name+'-left', btype=btype, obj_name=box_name, 
                gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,-np.pi/2,0), (-Xhalf,0,0))),
            BindingInfo(
                name=box_name+'-front', btype=btype, obj_name=box_name, 
                gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(np.pi/2,0,0), (0,-Yhalf,0))), 
            BindingInfo(
                name=box_name+'-back', btype=btype, obj_name=box_name, 
                gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(-np.pi/2,0,0), (0,Yhalf,0)))
            
        ]
    return bindings
    

In [8]:

class Scenario(Enum):
    single_object_single_robot = 0
    double_object_single_robot = 1
    single_object_dual_robot = 2
    assembly_3_piece = 3

In [9]:
class State:
    def __init__(self, node, obj_pos_dict, Q):
        self.node = node
        self.obj_pos_dict = obj_pos_dict
        self.Q = Q
    
    def get_tuple(self):
        return (self.node, self.obj_pos_dict, self.Q)
    
    def copy(self):
        return State(self.node, self.obj_pos_dict, self.Q)
        
    def __str__(self):
        return str((self.node, 
                    {k: str(np.round(v, 2)) for k, v in self.obj_pos_dict.items()} if self.obj_pos_dict is not None else None,  
                    str(np.round(self.Q, 2)) if self.Q is not None else None))
    
class SearchNode:
    def __init__(self, idx, state, parents, leafs, leafs_P, depth=None, edepth=None):
        self.idx, self.state, self.parents, self.leafs, self.leafs_P, self.depth, self.edepth = \
            idx, state, parents, leafs, leafs_P, depth, edepth

In [44]:

        
class ConstraintGraph:

    def __init__(self, urdf_path,  joint_names, link_names):
        self.urdf_path = urdf_path
        self.urdf_content = URDF.from_xml_file(urdf_path)
        self.link_info_list = get_link_info_list(link_names, self.urdf_content)
        self.robot_info = RobotInfo(self.link_info_list, rname = "rbt0", base_frame=np.identity(4,dtype=np.float32))
        self.joint_names = joint_names
        self.link_names = link_names
        
    def set_items(self, gitem_list, gframe_dict):
        self.gitem_list = gitem_list
        self.gframe_dict = gframe_dict
        
        # prepare ros
        self.pub, self.joints, self.rate_ros = get_publisher(self.joint_names)
        # prepare visualization markers
        self.marker_list = set_markers(self.gitem_list, self.gframe_dict, self.urdf_content)
        show_motion([np.array([0]*6)], self.marker_list, 
                    [[gframe_dict[mk.geometry.name] for mk in self.marker_list]], 
                    self.pub, self.joints, error_skip=1e-6, period=1e-6)
        
    def show_state(self, gframe_dict=None):
        if gframe_dict is not None:
            self.gframe_dict = gframe_dict
        show_motion([np.array([0]*6)], self.marker_list, 
                    [[gframe_dict[mk.geometry.name] for mk in self.marker_list]], 
                    self.pub, self.joints, error_skip=1e-6, period=1e-6)
        
    def set_binding(self, binfo_list):
        self.binfo_list = binfo_list
        
    def set_simulation(self, N_sim, binfo_list=None):
        self.N_sim = N_sim
        if binfo_list is not None:
            self.set_binding(binfo_list)
        self.mgraph = GraphModel(robot_info=self.robot_info, gitem_list=self.gitem_list, binfo_list=self.binfo_list, 
                           urdf_content=self.urdf_content, N_sim=self.N_sim)
        
    def update_handles(self):
        self.handle_dict = defaultdict(lambda: [])
        for k, v in self.mgraph.binding_dict.items():
            if not v.btype.is_binder():
                self.handle_dict[v.obj_layer.name] += [k]
        self.object_list = [name for name in self.mgraph.object_name_list if name in self.handle_dict]
        self.binder_list = [name for name in self.mgraph.binding_name_list if self.mgraph.binding_dict[name].btype.is_binder()]
        

    def get_unique_binders(self):
        uniq_binders = []
        for k_b, binder in self.mgraph.binding_dict.items():
            if binder.btype.is_binder() and not binder.btype.is_multiple():
                uniq_binders += [k_b]
        return uniq_binders

    def get_controlled_binders(self):
        controlled_binders = []
        for k_b, binder in self.mgraph.binding_dict.items():
            if binder.btype.is_binder() and binder.btype.is_controlled():
                controlled_binders += [k_b]
        return controlled_binders
    
    def build_graph(self):
        self.update_handles()
        self.node_dict = {}
        bindings = get_all_mapping(self.handle_dict.keys(), self.binder_list)
        handle_combinations = list(product(*[self.handle_dict[obj] for obj in self.object_list]))
        uniq_binders = self.get_unique_binders()
        ctrl_binders = self.get_controlled_binders()
        self.node_list = []
        self.node_dict = {}
        for binding in bindings:
            if all([np.sum([bd == ub for bd in  binding.values()])<=1 for ub in uniq_binders]):
                for hc in handle_combinations:
                    node = []
                    add_ok = True
                    for i_o, obj in zip(range(len(self.object_list)), self.object_list):
                        hndl_name = hc[i_o]
                        if self.mgraph.bind_cal.pair_condition(self.mgraph.binding_dict[binding[obj]], self.mgraph.binding_dict[hndl_name]):
                            node += [(obj, hc[i_o], binding[obj]) ]
                        else:
                            add_ok = False
                    if add_ok:
                        node = tuple(node)
                        self.node_list += [node]
                        self.node_dict[node] = []
        for node in self.node_list:
            fixed_in = [i_n for i_n in range(len(node)) if node[i_n][2] not in ctrl_binders]
            ctrl_in = [i_n for i_n in range(len(node)) if node[i_n][2] in ctrl_binders]
            if ctrl_in:
                bd_fixed = [node[idx] for idx in fixed_in]
                bd_ctrl = [node[idx][2] for idx in ctrl_in]
                pt_ctrl = [node[idx][1] for idx in ctrl_in]
                obj_ctrl = [node[idx][0] for idx in ctrl_in]
                nodes_same_fix = [nd for nd in  self.node_list if [nd[idx] for idx in fixed_in] == bd_fixed]
                nodes_diff_ctrl = [nd for nd in  nodes_same_fix if [nd[idx][2] for idx in ctrl_in] != bd_ctrl]
                nodes_diff_pt = [nd for nd in  nodes_diff_ctrl if [nd[idx][1] for idx in ctrl_in] != pt_ctrl]
                nodes_neighbor = [nd for nd in  nodes_diff_pt if len(set([bd[2] for bd, bd0 in zip(nd, node) if bd!=bd0]).intersection(bd_ctrl))==0]
                self.node_dict[node] += nodes_neighbor
                for nd_n in nodes_neighbor:
                    self.node_dict[nd_n] += [node]

        for node in self.node_dict.keys():
            if node in self.node_dict[node]:
                self.node_dict[node].remove(node)
            self.node_dict[node] = [node] + list(set(self.node_dict[node]))
        
        
        

In [45]:
cgraph = ConstraintGraph(urdf_path = URDF_PATH, joint_names = JOINT_NAMES, link_names = LINK_NAMES)
gtimer = GlobalTimer()

In [46]:
gitem_list, gframe_dict= get_link_items_offsets(color=(0,1,0,0.5), display=False)
gitem_list += [
    GeometryItem(name='box1', gtype=GeoType.BOX, dims=[0.1,0.1,0.1], color=(0.3,0.3,0.8,1), display=True, collision=True),
    GeometryItem(name='floor', gtype=GeoType.PLANE, dims=[3,3,0.01], color=(0.6,0.6,0.6,1), display=True, collision=True),
    GeometryItem(name='wall', gtype=GeoType.BOX, dims=[0.7,0.05,0.4], color=(0.4,0.3,0.0,1), display=True, collision=True),
    GeometryItem(name='stepper', gtype=GeoType.BOX, dims=[0.15,0.15,0.3], color=(0.4,0.3,0.0,1), display=True, collision=True),
    GeometryItem(name='goal_disp', gtype=GeoType.BOX, dims=[0.1,0.1,1e-3], color=(0.8,0.0,0.0,1), display=True, collision=False)
]
gframe_dict.update({"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,-0.2,0.050)), "world"),
                    "floor":GeometryFrame(SE3(Rot_zyx(0,0,0),(0,0,-0.00501)), "world"),
                    "wall":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.7,0.0,0.2)), "world"),
                    "stepper":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.4,0.4,0.15)), "world"),
                    "goal_disp":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.4,0.4,0.3)), "world")
                   })
binfo_list = [
    BindingInfo(
        name='vac1', btype=BindingType.VACC, obj_name='tool_mesh', gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,0), (0,0,0.05))), 
    BindingInfo(
        name='goal', btype=BindingType.PLANE, obj_name='stepper', gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,np.pi), (0,0,0.0751))), 
    BindingInfo(
        name='floortop', btype=BindingType.PLANE, obj_name='floor', gtype=None, Toff=SE3(Rot_zyx(0,0,np.pi), (0,0,0.005)))]

binfo_list +=  get_box_binding(get_by_name(gitem_list, "box1"))

In [47]:
cgraph.set_items(gitem_list, gframe_dict)
cgraph.set_binding(binfo_list)

publication OK
published: [0, 0, 0, 0, 0, 0]
publication OK - tool_mesh
publication OK - box1
publication OK - floor
publication OK - wall
publication OK - stepper
publication OK - goal_disp


In [48]:
cgraph.show_state(gframe_dict)

In [49]:
cgraph.set_simulation(N_sim=32)

In [50]:
cgraph.build_graph()

In [52]:
for k,v in cgraph.node_dict.items():
    print(k)
    print(v)
    print("="*100)

(('box1', 'box1-top', 'vac1'),)
[(('box1', 'box1-top', 'vac1'),), (('box1', 'box1-bottom', 'goal'),), (('box1', 'box1-bottom', 'floortop'),)]
(('box1', 'box1-bottom', 'vac1'),)
[(('box1', 'box1-bottom', 'vac1'),), (('box1', 'box1-top', 'goal'),), (('box1', 'box1-top', 'floortop'),)]
(('box1', 'box1-top', 'goal'),)
[(('box1', 'box1-top', 'goal'),), (('box1', 'box1-bottom', 'vac1'),)]
(('box1', 'box1-bottom', 'goal'),)
[(('box1', 'box1-bottom', 'goal'),), (('box1', 'box1-top', 'vac1'),)]
(('box1', 'box1-top', 'floortop'),)
[(('box1', 'box1-top', 'floortop'),), (('box1', 'box1-bottom', 'vac1'),)]
(('box1', 'box1-bottom', 'floortop'),)
[(('box1', 'box1-bottom', 'floortop'),), (('box1', 'box1-top', 'vac1'),)]


In [ ]:

for node in self.node_list:
    fixed_in = [i_n for i_n in range(len(node)) if node[i_n][2] not in ctrl_binders]
    ctrl_in = [i_n for i_n in range(len(node)) if node[i_n][2] in ctrl_binders]
    if ctrl_in:
        bd_fixed = [node[idx] for idx in fixed_in]
        bd_ctrl = [node[idx][2] for idx in ctrl_in]
        pt_ctrl = [node[idx][1] for idx in ctrl_in]
        obj_ctrl = [node[idx][0] for idx in ctrl_in]
        nodes_same_fix = [nd for nd in  self.node_list if [nd[idx] for idx in fixed_in] == bd_fixed]
        nodes_diff_ctrl = [nd for nd in  nodes_same_fix if [nd[idx][2] for idx in ctrl_in] != bd_ctrl]
        nodes_diff_pt = [nd for nd in  nodes_diff_ctrl if [nd[idx][1] for idx in ctrl_in] != pt_ctrl]
        nodes_neighbor = [nd for nd in  nodes_diff_pt if len(set([bd[2] for bd, bd0 in zip(nd, node) if bd!=bd0]).intersection(bd_ctrl))==0]
        self.node_dict[node] += nodes_neighbor
        for nd_n in nodes_neighbor:
            self.node_dict[nd_n] += [node]

for node in self.node_dict.keys():
    if node in self.node_dict[node]:
        self.node_dict[node].remove(node)
    self.node_dict[node] = [node] + list(set(self.node_dict[node]))

AttributeError: 'ConstraintGraph' object has no attribute 'binder_dict'

In [ ]:
#     graph.build_graph()
#     graph.search_graph_ordered_mp(initial_state = \
#                                   State((('box1','bottom_p','floor'),), 
#                                         {'box1': [0.5,-0.2,0.05,0,0,0,1]}, ZERO_JOINT_POSE),
#                                   goal_state = State((('box1','bottom_f','goal'),), None, None),
#                                   search_num=200, display=False, N_agents = 12,
#                                   terminate_on_first = True, tree_margin = 0,
#                                   depth_margin = 2, joint_motion_num=50, 
#                                   **dict(N=300, dt=0.01, vel_conv=1e-2, err_conv=1e-3))

In [ ]:

graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, binfo_list=binfo_list, urdf_content=urdf_content, 
                   N_sim=N_sim, learning_rate=5e-3, 
                   alpha_cl=0.00)
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all, Tbb_all = graph(None)
Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_objects, 1, 4,4))
res = graph.col_cal.calc_all(Tbo_all_res)

# Make Graph

In [8]:
N_sim = 32
N_joints = 9
DOF = 6


gtimer.tic("initialize")
binfo_list = [
    BindingInfo(
        name='vac1', btype=BindingType.VACC, obj_name='tool_mesh', gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,0), (0,0,0.05))), 
    BindingInfo(
        name='boxtop', btype=BindingType.SUCC, obj_name='box1', gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,0), (0,0,0.05))), 
    BindingInfo(
        name='boxbottom', btype=BindingType.SUCC, obj_name='box1', gtype=GeoType.SPHERE, Toff=SE3(Rot_zyx(0,0,np.pi), (0,0,-0.05))), 
    BindingInfo(
        name='floortop', btype=BindingType.PLANE, obj_name='floor', gtype=None, Toff=SE3(Rot_zyx(0,0,np.pi), (0,0,0.005)))]
graph = GraphModel(robot_info=robot_info, gitem_list=gitem_list, binfo_list=binfo_list, urdf_content=urdf_content, 
                   N_sim=N_sim, learning_rate=5e-3, 
                   alpha_cl=0.00)
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
T_all, Tbo_all, Tbb_all = graph(None)
Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_objects, 1, 4,4))
res = graph.col_cal.calc_all(Tbo_all_res)
gtimer.toc("initialize")
print(gtimer)

initialize: 	8768.0 ms/1 = 8768.133 ms 



In [9]:
gtimer.reset()
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
gtimer.tic("assign")
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
graph.set_slack_batch([[(0,1)]]*N_sim)
gtimer.toc("assign")
print(gtimer)

assign: 	6.0 ms/1 = 5.922 ms 



# Show rviz

In [20]:
Q_ = np.array([tuple(ZERO_JOINT_POSE+(np.random.rand(DOF)*2-1)*np.pi/100) for _ in range(N_sim)], dtype=np.float32)
gframe_dict.update({"box1":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,0.05)), "world"),
                    "floor":GeometryFrame(SE3(Rot_zyx(0,0,0),(0.5,0,-0.00501)), "world")
                   })
gframe_dict_list = [gframe_dict]*N_sim
graph.assign_Q(Q_)
graph.assign_frame_dict(gframe_dict_list)
graph.set_slack_batch([[(graph.binding_name_list.index('vac1'), 
                         graph.binding_name_list.index('boxtop'))]]+[[]]*(N_sim-1),
                     [[0]*graph.robot.DOF]*N_sim,
                     [[0]*graph.robot.DOF]+[[1]*graph.robot.DOF]+[[0]*graph.robot.DOF]*(N_sim-2))

In [21]:
Qcur = graph.get_Q()
q = np.array(Qcur[0])
gframevec = [gframe_dict[mk.geometry.name] for mk in marker_list]
pose_list = [q]
gframevec_list = [gframevec]
show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)

# loop display

In [22]:
gtimer.reset()
lpf = LowPassFilter(0.5)
for _ in range(100):
    gtimer.tic("calc")
    Qcur = graph.get_Q()
    T_all, Tbo_all, Tbb_all = graph(graph.binding_index_list)
    jac_r, jac_o, jac_b, jac_brot = graph.jacobian(T_all, Tbo_all, Tbb_all)
    Tbo_all_res = tf.reshape(Tbo_all, (graph.N_sim, 1, graph.num_objects, 1, 4,4))
    dist_all, flag_all, vec_all, mask_all = graph.col_cal.calc_all(Tbo_all_res)
    jac_d = graph.col_cal.jacobian_distance(jac_o, vec_all)
    Tbb_all_res = tf.reshape(Tbb_all, (graph.N_sim, 1, graph.num_binding, 1, 4,4))
    b_dist_all, flag_all, vec_all, b_mask_all, angle_all, vec_angle, mask_rot = graph.bind_cal.calc_all(Tbb_all_res)
    mask_rot = tf.expand_dims(graph.bind_cal.mask_rot, axis=-1)
    b_dist_masked = b_dist_all*mask_rot
    jac_bind, jac_ang = graph.bind_cal.jacobian_binding(jac_b, vec_all, jac_brot, vec_angle)
    jac_bind_masked = jac_bind*mask_rot
    jac_bind_tr = tf.transpose(jac_bind_masked, (0,2,1))
    jac_bind_inv = tf.matmul(jac_bind_tr, tf.linalg.inv(tf.matmul(jac_bind_masked,jac_bind_tr)+graph.bind_cal.mask_rot_diag_rev))
    jac_ang_stack = tf.gather_nd(jac_ang, [graph.bind_cal.pair_axis_list]*graph.N_sim, batch_dims=1)
    jac_ang_masked = jac_ang_stack*mask_rot
    jac_ang_tr = tf.transpose(jac_ang_masked, (0,2,1))
    jac_ang_inv = tf.matmul(jac_ang_tr, tf.linalg.inv(tf.matmul(jac_ang_masked,jac_ang_tr)+graph.bind_cal.mask_rot_diag_rev))
    angle_all_stack = tf.gather_nd(angle_all, [graph.bind_cal.pair_axis_list]*graph.N_sim, batch_dims=1)
    angle_all_masked = angle_all_stack*mask_rot
    dQ = (
        K.sum(
            tf.matmul(jac_bind_inv,b_dist_masked) 
            + tf.matmul(jac_ang_inv,angle_all_masked), axis=-1)
        + graph.joint_mask_batch * (graph.joint_batch - Qcur))*graph.rate_update
    dQ_clip = clip_gradient_elem_wise(dQ, graph.dQ_max)
    dQ_pre = lpf(dQ_clip)

    
    #cut collision
    dD = K.sum(jac_d*tf.expand_dims(dQ_clip, axis=-2), axis=-1, keepdims=True)
    Dcur = dist_all
    mask_colliding = tf.cast(Dcur+dD<0,dtype=tf.float32)
    sign_jac_d =tf.sign(jac_d)
    dQ_cut = (mask_colliding* mask_all)*(Dcur+dD)*jac_d/(K.sum(tf.square(jac_d), axis=-1, keepdims=True)+1e-16)
    dQ_cut = K.sum(dQ_cut, axis=-2) # need to implement optimizer
    
    Qnew = Qcur+dQ_pre+dQ_cut

    gtimer.toc("calc")
    gtimer.tic("assign")
    graph.assign_Q(Qnew)
    gtimer.toc("assign")

    pose_list = [np.array(graph.get_Q()[0])]
    gframevec_list = [[gframe_dict[mk.geometry.name] for mk in marker_list]]
    show_motion(pose_list, marker_list, gframevec_list, pub, joints, error_skip=1e-6, period=1e-6)
    timer.sleep(0.05)
print(gtimer)

calc: 	4848.0 ms/100 = 48.475 ms 
assign: 	51.0 ms/100 = 0.512 ms 



In [13]:
gframe_dict_list = graph.update_slack_frames(gframe_dict_list, T_all, Tbo_all)